In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Cargar datos 
archivo_excel = 'Columna.csv'
data = pd.read_csv(archivo_excel)
data

,text_punc_num
0,la peor delincuencia no es el de las calles e...
1,la policaa nacional rechaza y condena el acto...
2,ayer en villavicencio mientras daba una charl...
3,cuando va a colocar una unidad movil de maner...
4,tal vez no sea buena idea dejarse asesorar po...
...,...
7900,el manido chiste de la guitarra y la utilizac...
7901,el manido chiste de la guitarra y la utilizac...
7902,extradeportes asisergio lopez mediocentro y ...
7903,francisco chahuan hoy llego a la moneda con u...


In [2]:
replacements = {
    'é': 'e',
    'ñ': 'n',
    'í': 'i',
    'ó': 'o',
    'á': 'a',
    'ú': 'u',
    'ã³':'o',
    'ðÿ':' ',
    'š½ ':'si'
    # Añade más reemplazos según sea necesario
}

for original, replacement in replacements.items():
    data['text_punc_num'] = data['text_punc_num'].str.replace(original, replacement)

In [3]:
# nombre_archivo_nuevo = 'Columna.csv'
# data['text_punc_num'].to_csv(nombre_archivo_nuevo, index=False)

In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Descargar las stopwords si es necesario
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Preprocesamiento del texto
def preprocesar(texto):

    if isinstance(texto, float):
        return []
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar caracteres no alfabéticos
    texto = re.sub(r'[^a-záéíóúñ]+', ' ', texto)
    # Tokenización
    palabras = word_tokenize(texto)
    # Eliminar stopwords
    palabras = [palabra for palabra in palabras if palabra not in stopwords.words('spanish')]
    # Lematización (opcional)
#     lemmatizer = WordNetLemmatizer()
#     palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras]
    return palabras

# Ejemplo de uso
# texto_ejemplo = "Este es un ejemplo de texto para preprocesar."
# texto_procesado = preprocesar(texto_ejemplo)
# print(texto_procesado)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguelsamaniego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/miguelsamaniego/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/miguelsamaniego/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
data['texto_procesado'] = data['text_punc_num'].apply(preprocesar)
data['texto_procesado']

0       [peor, delincuencia, calles, prensa, roba, ver...
1       [policaa, nacional, rechaza, condena, acto, de...
2       [ayer, villavicencio, mientras, daba, charla, ...
3       [va, colocar, unidad, movil, manera, permanent...
4       [tal, vez, buena, idea, dejarse, asesorar, exm...
                              ...                        
7900    [manido, chiste, guitarra, utilizacion, delinc...
7901    [manido, chiste, guitarra, utilizacion, delinc...
7902    [extradeportes, asisergio, lopez, mediocentro,...
7903    [francisco, chahuan, hoy, llego, moneda, guita...
7904    [solo, delito, podido, probar, aa, n, detracto...
Name: texto_procesado, Length: 7905, dtype: object

In [6]:
from gensim.corpora import Dictionary

# Crear un diccionario a partir de los textos procesados
diccionario = Dictionary(data['texto_procesado'])

# Crear un corpus (representación BOW)
corpus = [diccionario.doc2bow(texto) for texto in data['texto_procesado']]


In [7]:
from gensim.models import LdaModel

# Número de temas que quieres identificar
num_temas = 5

# Crear y entrenar el modelo LDA
lda = LdaModel(corpus=corpus, id2word=diccionario, num_topics=num_temas, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Mostrar los temas
lda.show_topics()


[(0,
  '0.088*"delincuencia" + 0.023*"fanb" + 0.023*"tregua" + 0.018*"organizada" + 0.018*"gobierno" + 0.013*"ahora" + 0.012*"da" + 0.012*"combatir" + 0.012*"cara" + 0.011*"sistema"'),
 (1,
  '0.075*"delincuencia" + 0.071*"corrupcion" + 0.068*"dicen" + 0.064*"maaana" + 0.063*"polaticos" + 0.062*"ligados" + 0.062*"parapolatica" + 0.062*"clanes" + 0.062*"marchoporpetro" + 0.061*"hoa"'),
 (2,
  '0.080*"guitarra" + 0.043*"delincuencia" + 0.028*"rn" + 0.028*"hoy" + 0.027*"moneda" + 0.026*"cosa" + 0.026*"dice" + 0.026*"boric" + 0.025*"mala" + 0.025*"llego"'),
 (3,
  '0.053*"delincuencia" + 0.026*"n" + 0.021*"si" + 0.015*"solo" + 0.015*"colombia" + 0.014*"desatada" + 0.013*"familia" + 0.011*"sabemos" + 0.011*"judicial" + 0.011*"lopez"'),
 (4,
  '0.075*"presidente" + 0.029*"mientras" + 0.026*"dijo" + 0.023*"ayer" + 0.022*"libro" + 0.022*"joven" + 0.022*"pidio" + 0.022*"delina" + 0.021*"daba" + 0.021*"villavicencio"')]

In [8]:
# Asumiendo que 'lda' es tu modelo LDA
num_palabras = 10  # Número de palabras clave que deseas mostrar por tema

for i, topic in enumerate(lda.show_topics(num_topics=num_temas, num_words=num_palabras, formatted=False)):
    palabras_clave = [word for word, _ in topic[1]]
    print(f"Tema {i + 1}: {' - '.join(palabras_clave)}")


Tema 1: delincuencia - fanb - tregua - organizada - gobierno - ahora - da - combatir - cara - sistema
Tema 2: delincuencia - corrupcion - dicen - maaana - polaticos - ligados - parapolatica - clanes - marchoporpetro - hoa
Tema 3: guitarra - delincuencia - rn - hoy - moneda - cosa - dice - boric - mala - llego
Tema 4: delincuencia - n - si - solo - colombia - desatada - familia - sabemos - judicial - lopez
Tema 5: presidente - mientras - dijo - ayer - libro - joven - pidio - delina - daba - villavicencio


In [9]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Preparar los datos para pyLDAvis
lda_display = gensimvis.prepare(lda, corpus, diccionario, sort_topics=False)

# Mostrar la visualización
pyLDAvis.display(lda_display)


En la parte izquierda de la imagen, tienes un "Mapa de Distancia Intertópica" que ha sido generado a través de un proceso de escalamiento multidimensional (multidimensional scaling, MDS). Este mapa visualiza la distancia entre distintos tópicos, lo que puede interpretarse como qué tan diferentes son los tópicos entre sí. Los números representan los tópicos identificados, y el tamaño de los círculos suele corresponder a la proporción de documentos en el conjunto de datos que pertenecen a ese tópico. Los círculos que están más cerca entre sí representan tópicos que comparten una similitud más significativa en términos de las palabras que contienen.

En la parte derecha, tienes un gráfico de barras que muestra los "30 términos más sobresalientes" del tópico seleccionado (en este caso parece ser el tópico 1, aunque no está explícitamente marcado). Cada barra representa la frecuencia estimada de un término dentro del tópico seleccionado, lo que ayuda a entender cuáles son las palabras clave que caracterizan ese tópico. Estos términos son importantes para interpretar el tema subyacente del tópico. Por ejemplo, palabras como "guitarra" y "corrupción" pueden sugerir temas relacionados con música y política, respectivamente, pero sin más contexto es difícil definir el tema exacto.

La relevancia de los términos, que también parece estar anotada en el gráfico, es una medida calculada que combina la frecuencia del término con su exclusividad en el tópico, ayudando a identificar palabras que no solo son frecuentes sino también distintivas del tópico en cuestión.

